In [ ]:
# import numpy
import pandas

df = pandas.read_excel(
    "C:/Users/Nilay.Baranwal/Desktop/PanIndia/PanIndia_3500.xlsb", engine="pyxlsb"
)

print(
    # df.head(10000).to_csv(
    df.to_csv("C:/Users/Nilay.Baranwal/Desktop/PanIndia/PanIndia_3500.csv", index=False)
)

In [5]:
# import pprint
import pandas
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import threading

root = tk.Tk()
root.title("Processor")
root.geometry("570x80")
root.resizable(width=False, height=False)
path_var = tk.StringVar()


def process(prog):
    path = path_var.get()
    path_var.set("")
    if len(path) == 0:
        # err_popup = tk.Toplevel(root)
        # err_popup.title("Error")
        # err_popup.geometry("200x35")
        # err_popup.resizable(width=False, height=False)
        # ttk.Label(
        #     err_popup,
        #     text="Path cannot be Empty.",
        #     font=("Segoe UI Semibold", 10),
        #     justify="center",
        # ).pack()
        messagebox.showerror("Error", "Path cannot be Empty.")
    else:
        df = pandas.read_csv(path)

        new_column = []
        for i, row in df.iterrows():
            tmp = row["Area"]
            cnt = 0
            for j, duplRow in df.iloc[i:].iterrows():
                if j != i and duplRow["Label"] != row["Label"]:
                    break
                if j != i and duplRow["Label"] == row["Label"]:
                    tmp += duplRow["Area"]
                    cnt += 1
            if cnt != 0:
                new_column.append(tmp)
            else:
                new_column.append("NULL")

        df.insert(7, "Area_Sum", new_column)
        df = df.drop_duplicates(subset=["Label"])
        df = df.reset_index(drop=True)
        df["Area"] = df["Area"].mask(df["Area_Sum"] != "NULL", df["Area_Sum"])
        df = df.drop(columns=["Area_Sum"])
        df["Area"] = df["Area"].astype(float)

        percent_diff = []
        avg_list = []
        i = 0
        # for i in range(0, df["Index"].iloc[-1]):
        while i <= df["Index"].iloc[-1]:
            cnt = 0
            for j in range(i, df["Index"].iloc[-1] + 1):
                if df["Sap_Id"].iloc[j] != df["Sap_Id"].iloc[i]:
                    break
                else:
                    cnt += 1
            diff = []
            if cnt > 0:
                diff.extend(df.iloc[i : i + cnt].nlargest(cnt, "Area")["Area"].tolist())
            avg = (diff[0] + diff[1]) / 2 if len(diff) >= 2 else diff[0]
            # avg_list.extend((avg, None, None))
            incr = (
                diff[2] * 100 / avg
                if len(diff) >= 3
                else "ERROR: >= 3 Sectors Required"
            )
            # percent_diff.extend((round(incr, 2), None, None))
            for j in range(0, cnt):
                if j == df["Area"].iloc[i : i + cnt].idxmin() - i:
                    avg_list.append(avg)
                    percent_diff.append(
                        incr if isinstance(incr, str) else round(incr, 2)
                    )
                else:
                    avg_list.append(None)
                    percent_diff.append(None)
            i += cnt

        df.insert(7, "Avg. b/w Greatest 2 Areas", avg_list)
        df.insert(8, "% Difference b/w Avg. & Smallest", percent_diff)
        print(
            df.to_csv(
                path[:-4] + "_Processed.csv",
                index=False,
            )
        )

    prog.set("")


def process_btn():
    path_entry.config(state="disabled")
    browse_btn.config(state="disabled")
    sub_btn.config(state="disabled")
    progress = ttk.Progressbar(root)
    progress.grid(row=1, column=1, pady=10)
    prog = tk.StringVar()
    threading.Thread(target=process, args=(prog,), daemon=True).start()
    progress.start(10)
    root.wait_variable(prog)
    progress.destroy()
    path_entry.config(state="normal")
    browse_btn.config(state="normal")
    sub_btn.config(state="normal")


def file_path():
    path_entry.insert(0, filedialog.askopenfilename())


path_label = ttk.Label(
    root,
    text="Source File Path (Use '/' instead of '\\'): ",
    font=("Segoe UI Semibold", 10),
)
path_entry = ttk.Entry(
    root, width=35, textvariable=path_var, font=("Segoe UI Semibold", 10)
)

browse_btn = ttk.Button(
    root,
    text="Browse",
    command=file_path,
)

sub_btn = ttk.Button(
    root,
    text="Process File",
    command=process_btn,
    # command=threading.Thread(target=submit).start,
    # font=("Segoe UI Semibold", 10),
)

path_label.grid(row=0, column=0, pady=5)
path_entry.grid(row=0, column=1, pady=5)
browse_btn.grid(row=0, column=2, padx=5, pady=5)
sub_btn.grid(row=1, column=0, pady=10)

root.mainloop()